## Vamos trabalhar com os dados abertos do ONS - Operador Nacional do Sistema Elétrico.

### Optei por trabalhar com os dados de Carga de Energia. Definição conforme site: [Dados de carga por subsistema numa data de referência em base diária.](https://dados.ons.org.br/dataset/carga-energia)

### Nessa etapa, iremos criar novas colunas a partir das informações já existentes. O objetivo é que as novas variáveis auxiliem nas futuras análises

### Load das bibliotecas e definição do contexto spark.

In [1]:
import pyspark

from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
sc = SparkContext.getOrCreate()

spark = (SparkSession
      .builder
      .getOrCreate()
     )

print('ApplicationID:', sc.applicationId)

ApplicationID: local-1631045312067


### Caminhos de Leitura e Escrita das Bases de dados

In [3]:
read_path = '../data/refined/02_transform/'

write_path = '../data/refined/03_enriquecimento'

In [4]:
df = spark.read.parquet(read_path)

print(f'Número de registros: {df.count()}')

df.printSchema()

Número de registros: 31649
root
 |-- id_subsistema: byte (nullable = true)
 |-- nom_subsistema: string (nullable = true)
 |-- din_instante: date (nullable = true)
 |-- val_cargaenergiamwmed: double (nullable = true)



### A primeira variável proposta é o ano do registro. Essa variável facilita análises temporais.

In [5]:
df = df.withColumn('ano', F.year(F.col('din_instante')).cast(IntegerType()))

### A segunda é o mês do registro. Muito parecida com a anterior, facilita a comparação entre diferentes meses dentro de um mesmo ano, ou de um mesmo mês para diferentes anos.

In [6]:
df = df.withColumn('mes', F.month(F.col('din_instante')).cast(IntegerType()))

### Propões criar um 'id_medicao' único para cada registro. Os dados serão organizados em ordem crescente baseado em 'din_instante'. O critério de desempate para cada data será o 'id_subsistema'.

In [7]:
window_spec = Window.orderBy(F.col('din_instante'), F.col('id_subsistema'))
df = df.withColumn('id_medicao', F.row_number().over(window_spec))

### Escrita da base com reordenação das colunas

In [8]:
df_variaveis = ['id_medicao', 
             'nom_subsistema', 
             'id_subsistema',
             'val_cargaenergiamwmed',
             'din_instante',
             'mes',
             'ano',]

In [9]:
df.select(df_variaveis).write.parquet(write_path)

In [10]:
sc.stop()